In [109]:
### Analysis for consumer finance company

In [110]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [111]:
# read the data set we have.

In [112]:
loan_data = pd.read_csv('loan.csv')

In [113]:
# As part of initial step, understand the data set given

In [114]:
loan_data.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599,5000,5000,4975.0,36 months,10.65%,162.87,B,B2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,1077430,1314167,2500,2500,2500.0,60 months,15.27%,59.83,C,C4,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,1077175,1313524,2400,2400,2400.0,36 months,15.96%,84.33,C,C5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,1076863,1277178,10000,10000,10000.0,36 months,13.49%,339.31,C,C1,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
4,1075358,1311748,3000,3000,3000.0,60 months,12.69%,67.79,B,B5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


In [115]:
loan_data.shape

(39717, 111)

In [116]:
loan_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39717 entries, 0 to 39716
Columns: 111 entries, id to total_il_high_credit_limit
dtypes: float64(74), int64(13), object(24)
memory usage: 33.6+ MB


In [117]:
## from the above data from head, looks like tehre are lot of NAs 
## see if there are any columns that are completely empty 

In [118]:
nan_cols = [i for i in loan_data.columns if loan_data[i].isnull().all()]
len(nan_cols)

54

In [119]:
#confirm the above data by looking at isna for each column
loan_data.isna().sum()

id                                0
member_id                         0
loan_amnt                         0
funded_amnt                       0
funded_amnt_inv                   0
                              ...  
tax_liens                        39
tot_hi_cred_lim               39717
total_bal_ex_mort             39717
total_bc_limit                39717
total_il_high_credit_limit    39717
Length: 111, dtype: int64

In [120]:
# There are some 54 columns that are completely empty.
# We need to remove these columns
#just to be on a safer side, look at few columns randomly in the csv


In [121]:
loan_data.dropna(axis=1, how='all', inplace=True)

In [122]:
#confirm if these columns are removed.
loan_data.shape

(39717, 57)

In [123]:
# Still there are columns which have more than 90% missing data in them
# lets see what all columns have more than 90% missing data                          

In [124]:
P90 = 39717*0.90
nan90_cols = [i for i in loan_data.columns if loan_data[i].isna().sum()>=P90]
nan90_cols

['mths_since_last_record', 'next_pymnt_d']

In [125]:
# Further looks like there are some columns where there are more zeros, let's see if there are any columns where all values represent same data

In [126]:
cols_nounique = [i for i in loan_data.columns if loan_data[i].nunique()==1]
cols_nounique

['pymnt_plan',
 'initial_list_status',
 'collections_12_mths_ex_med',
 'policy_code',
 'application_type',
 'acc_now_delinq',
 'chargeoff_within_12_mths',
 'delinq_amnt',
 'tax_liens']

In [127]:
# since all values are same, it doesn't provide any valuable insight.
# drop these columns as well

In [128]:
loan_data.drop(columns=cols_nounique, inplace=True, axis=1)

In [129]:
# verify the drop again

In [130]:
loan_data.shape

(39717, 48)

In [131]:
loan_data_cols = loan_data.columns.tolist()

In [142]:
# save the filtered data into new csv so that it is easy to look at data
loan_data.to_csv('FilteredLoanData.csv', index=False)

In [ ]:
#Let's see manually if any column is further redundant

In [ ]:
#columns that are ot necessary
# 1. url - we only have loan id changing. hence this can be ignored
# 2. desc - we cannot make any insight out of this hence drop
# 3. title- we cannot make any insight out of this hence drop
# 4. dlinq 2yrs - Derived column from mths since last delinq
# 5. It is possible for a person whose loan got ended in the past, and the one who defaulted to have last payment date in past,
# we cannot make any insights out of this, as we can't distinguish well with the following columns.
# last_pymnt_d, last_pymnt_amnt, next_pymntd
# Hence Drop

In [143]:
loan_data.shape

(39717, 41)

In [144]:
loan_data.drop(columns=['url','desc','title','delinq_2yrs','last_pymnt_d','last_pymnt_amnt','next_pymnt_d'], inplace=True)

KeyError: "['url' 'desc' 'title' 'delinq_2yrs' 'last_pymnt_d' 'last_pymnt_amnt'\n 'next_pymnt_d'] not found in axis"

In [145]:
loan_data.shape

(39717, 41)

In [146]:
loan_data.columns

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'issue_d', 'loan_status', 'purpose', 'zip_code', 'addr_state', 'dti',
       'earliest_cr_line', 'inq_last_6mths', 'mths_since_last_delinq',
       'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal',
       'revol_util', 'total_acc', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_credit_pull_d', 'pub_rec_bankruptcies'],
      dtype='object')

In [147]:
# we are done with investigation on important columns, now we have to perform data cleaning on rows.

In [148]:
loan_data.dropna(axis=0,how='all',inplace=True)

In [149]:
#lets see the columns having NAs
[i for i in loan_data.columns if loan_data[i].isna().sum()> 0]

['emp_title',
 'emp_length',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'revol_util',
 'last_credit_pull_d',
 'pub_rec_bankruptcies']

In [150]:
loan_data.emp_title.fillna('',inplace=True)

# apply(lambda x: "" if np.isnan() else str(x)) 

In [151]:
loan_data.emp_length.value_counts()

10+ years    8879
< 1 year     4583
2 years      4388
3 years      4095
4 years      3436
5 years      3282
1 year       3240
6 years      2229
7 years      1773
8 years      1479
9 years      1258
Name: emp_length, dtype: int64

In [152]:
loan_data.emp_length.isna().sum()

1075

In [153]:
# since emp_length is a categorical variable there cannot be a mean or median so we shall ignore missing values

In [154]:
loan_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39717 entries, 0 to 39716
Data columns (total 41 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       39717 non-null  int64  
 1   member_id                39717 non-null  int64  
 2   loan_amnt                39717 non-null  int64  
 3   funded_amnt              39717 non-null  int64  
 4   funded_amnt_inv          39717 non-null  float64
 5   term                     39717 non-null  object 
 6   int_rate                 39717 non-null  object 
 7   installment              39717 non-null  float64
 8   grade                    39717 non-null  object 
 9   sub_grade                39717 non-null  object 
 10  emp_title                39717 non-null  object 
 11  emp_length               38642 non-null  object 
 12  home_ownership           39717 non-null  object 
 13  annual_inc               39717 non-null  float64
 14  verification_status   

Now lets get into formatting the data

In [ ]:
'''
Convert Objects into respective data types
term : drop months keyword
int rate: .remove percent
issue_d datetime
zipcode remove xx
earliest_cr_line datetime
revol_util remove percent
total_pymnt and totalpymnt inv avg ratio of diff
last_credit_pull_d datetime
'''

In [155]:
# since there are only two values this is categorical data type, we can keep it as it is.
loan_data.term.value_counts()

 36 months    29096
 60 months    10621
Name: term, dtype: int64

In [157]:
# Convert int rate to numeric values
loan_data.int_rate.apply(lambda x: float(str(x).rstrip('%')))

0        10.65
1        15.27
2        15.96
3        13.49
4        12.69
         ...  
39712     8.07
39713    10.28
39714     8.07
39715     7.43
39716    13.75
Name: int_rate, Length: 39717, dtype: float64

In [163]:
#Strip zipCode extras
loan_data.zip_code.apply(lambda x: int(str(x).rstrip('xx')))

0        860
1        309
2        606
3        917
4        972
        ... 
39712    802
39713    274
39714     17
39715    208
39716     27
Name: zip_code, Length: 39717, dtype: int64

In [165]:
(loan_data.total_pymnt - loan_data.total_pymnt_inv).describe()

count    39717.000000
mean       586.447426
std       2174.798916
min         -0.005000
25%          0.000000
50%          0.004961
75%        172.999740
max      45431.040890
dtype: float64

In [170]:
(loan_data.total_pymnt_inv/loan_data.total_pymnt).quantile(0.1)

0.9156004564668956

In [174]:
#Convert the columns to corresponding dates
loan_data.issue_d = pd.to_datetime(loan_data.issue_d,format='%b-%y')

In [175]:
loan_data.issue_d

0       2011-12-01
1       2011-12-01
2       2011-12-01
3       2011-12-01
4       2011-12-01
           ...    
39712   2007-07-01
39713   2007-07-01
39714   2007-07-01
39715   2007-07-01
39716   2007-06-01
Name: issue_d, Length: 39717, dtype: datetime64[ns]

In [176]:
loan_data.earliest_cr_line = pd.to_datetime(loan_data.earliest_cr_line,format='%b-%y')

In [177]:
loan_data.earliest_cr_line

0       1985-01-01
1       1999-04-01
2       2001-11-01
3       1996-02-01
4       1996-01-01
           ...    
39712   1990-11-01
39713   1986-12-01
39714   1998-10-01
39715   1988-11-01
39716   2003-10-01
Name: earliest_cr_line, Length: 39717, dtype: datetime64[ns]

In [178]:
loan_data.last_credit_pull_d = pd.to_datetime(loan_data.last_credit_pull_d,format='%b-%y')

In [179]:
loan_data.last_credit_pull_d

0       2016-05-01
1       2013-09-01
2       2016-05-01
3       2016-04-01
4       2016-05-01
           ...    
39712   2010-06-01
39713   2010-07-01
39714   2007-06-01
39715   2007-06-01
39716   2010-06-01
Name: last_credit_pull_d, Length: 39717, dtype: datetime64[ns]